# Import librairies

In [ ]:
# Install nltk
!pip install nltk

In [23]:
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [24]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/xavier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/xavier/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/xavier/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Import des données

In [63]:
airline = pd.read_csv("../raw_data/Tweets.csv")
#df = df['text','airline_sentiment']

In [64]:
airline.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [65]:
airline = df[['tweet_id','text','airline_sentiment']]
airline.head()

,tweet_id,text,airline_sentiment
0,570306133677760513,@VirginAmerica What @dhepburn said.,neutral
1,570301130888122368,@VirginAmerica plus you've added commercials t...,positive
2,570301083672813571,@VirginAmerica I didn't today... Must mean I n...,neutral
3,570301031407624196,@VirginAmerica it's really aggressive to blast...,negative
4,570300817074462722,@VirginAmerica and it's a really big bad thing...,negative


In [66]:
airline.isna().sum().sort_values(ascending=False) #NaN count for each column

tweet_id             0
text                 0
airline_sentiment    0
dtype: int64

In [67]:
airline.rename({'tweet_id':'textID', 'text':'text', 'airline_sentiment':'sentiment'}, axis='columns', inplace = True)
airline["selected_text"] = 'from airline'
airline = airline[['textID','text', 'selected_text','sentiment']]
airline.head()

,textID,text,selected_text,sentiment
0,570306133677760513,@VirginAmerica What @dhepburn said.,from airline,neutral
1,570301130888122368,@VirginAmerica plus you've added commercials t...,from airline,positive
2,570301083672813571,@VirginAmerica I didn't today... Must mean I n...,from airline,neutral
3,570301031407624196,@VirginAmerica it's really aggressive to blast...,from airline,negative
4,570300817074462722,@VirginAmerica and it's a really big bad thing...,from airline,negative


In [72]:
airline['sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: sentiment, dtype: int64

In [73]:
airline = airline[airline.sentiment != "neutral"]

In [74]:
airline["sentiment"] = airline["sentiment"].map({'positive' : 1, "negative" : 0})
airline.head()

,textID,text,selected_text,sentiment
1,570301130888122368,@VirginAmerica plus you've added commercials t...,from airline,1
3,570301031407624196,@VirginAmerica it's really aggressive to blast...,from airline,0
4,570300817074462722,@VirginAmerica and it's a really big bad thing...,from airline,0
5,570300767074181121,@VirginAmerica seriously would pay $30 a fligh...,from airline,0
6,570300616901320704,"@VirginAmerica yes, nearly every time I fly VX...",from airline,1


In [68]:
train = pd.read_csv("../raw_data/train.csv")

In [75]:
train = train[train.sentiment != "neutral"]

In [76]:
train["sentiment"] = train["sentiment"].map({'positive' : 1, "negative" : 0})

In [77]:
train.head()

,textID,text,selected_text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,0
2,088c60f138,my boss is bullying me...,bullying me,0
3,9642c003ef,what interview! leave me alone,leave me alone,0
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",0
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,1


In [78]:
data = pd.concat([airline, train], axis=0)

In [80]:
data.head()

,textID,text,selected_text,sentiment
1,570301130888122368,@VirginAmerica plus you've added commercials t...,from airline,1
3,570301031407624196,@VirginAmerica it's really aggressive to blast...,from airline,0
4,570300817074462722,@VirginAmerica and it's a really big bad thing...,from airline,0
5,570300767074181121,@VirginAmerica seriously would pay $30 a fligh...,from airline,0
6,570300616901320704,"@VirginAmerica yes, nearly every time I fly VX...",from airline,1


In [81]:
data.shape

(27904, 4)

In [57]:
def preprocessing(text):
    punctuations = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for punctuation in punctuations:
        text = text.replace(punctuation, '')
    text = ''.join(word for word in text if not word.isdigit())
    
    return text.lower()

In [58]:
df['preprocessed_text'] = df['text'].map(preprocessing)
df.head()

,textID,text,selected_text,sentiment,preprocessed_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,id have responded if i were going
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad i will miss you here in san diego
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,sons of why couldnt they put them on the rel...


In [39]:
# Tokenizing and Remove stopwords
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

# Fonctions à encapsuler
def rm_sw(text):
    
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text)
    text = [w for w in word_tokens if not w in stop_words]
    
    return text

In [40]:
df['preprocessed_text'] = df['text'].map(rm_sw)
df.head()

,textID,text,selected_text,sentiment,preprocessed_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[I, `, responded, ,, I, going]"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[Sooo, SAD, I, miss, San, Diego, !, !, !]"
2,088c60f138,my boss is bullying me...,bullying me,negative,"[boss, bullying, ...]"
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[interview, !, leave, alone]"
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[Sons, *, *, *, *, ,, `, put, releases, alread..."


In [42]:
# Stemming
#from nltk.stem.porter import PorterStemmer
#def stemming(text):
#    stemmer = PorterStemmer()
#    text = [stemmer.stem(word) for word in text]
#    
#    return text

In [43]:
#df['preprocessed_text'] = df['text'].map(stemming)

In [47]:
df.head()

,textID,text,selected_text,sentiment,preprocessed_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I ` d h a v e r e s p o n d e d , i f ..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,S o o o S A D I w i l l m i s s y ...
2,088c60f138,my boss is bullying me...,bullying me,negative,m y b o s s i s b u l l y i n g m e . . .
3,9642c003ef,what interview! leave me alone,leave me alone,negative,w h a t i n t e r v i e w ! l e a v e ...
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"S o n s o f * * * * , w h y c o u l ..."


In [31]:
df.head()

,tweet_id,text,airline_sentiment
0,570306133677760513,@VirginAmerica What @dhepburn said.,neutral
1,570301130888122368,@VirginAmerica plus you've added commercials t...,positive
2,570301083672813571,@VirginAmerica I didn't today... Must mean I n...,neutral
3,570301031407624196,@VirginAmerica it's really aggressive to blast...,negative
4,570300817074462722,@VirginAmerica and it's a really big bad thing...,negative


In [44]:
# Fonctions à encapsuler
from nltk.stem import WordNetLemmatizer

def lemmatize(text):
    
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lemmatized)
    return text

In [45]:
df['preprocessed_text'] = df['text'].map(lemmatize)

In [48]:
df.head()

,textID,text,selected_text,sentiment,preprocessed_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I ` d h a v e r e s p o n d e d , i f ..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,S o o o S A D I w i l l m i s s y ...
2,088c60f138,my boss is bullying me...,bullying me,negative,m y b o s s i s b u l l y i n g m e . . .
3,9642c003ef,what interview! leave me alone,leave me alone,negative,w h a t i n t e r v i e w ! l e a v e ...
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"S o n s o f * * * * , w h y c o u l ..."


# New heading

# Vectorizing - N grams

In [51]:
# N-grams and tf idf
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(ngram_range = (2,2))
X_vec = tf_idf_vectorizer.fit_transform(df['preprocessed_text'])
X_vec

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
# Modèle simple - MultinomialNB
from sklearn.model_selection import cross_validate
model = MultinomialNB()
y = df['sentiment']
cv_results = cross_validate(model, X_tfidf, y, scoring='accuracy', cv=5)
score = cv_results['test_score'].mean()

In [ ]:
# Pipeline Modèle
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

X= df['preprocessed_text']
y= df['sentiment']

# Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB()),
                ])

# Set parameters to search (model and vectorizer)
parameters = {
    'tfidf__ngram_range': ((1,1), (2,2)),
    'nb__alpha': (0.1,1),
            }

# Perform grid search on pipeline
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, 
                           verbose=1, scoring = "accuracy", 
                           refit=True, cv=5)

grid_search.fit(X,y)

In [ ]:
grid_search.best_params_
grid_search.best_score_